# Modelo Preditivo de Fraude - Seletor de Evento de POC

## Objetivo
Este análise tem por objetivo selecionar dentre todos os eventos que ocorrem em ATMs aqueles que serão usados como features no modelo preditivo.
São seelcionados os eventos que ocorrem acima da ocorrência padrão para a Agência/ATM diariamente em dias de POCs reportados, não registrandoocorrências diárias fora do padrão nos demais dias.
Sâo usados os primeiros 30 dias dos 90 dias considerados para gerar os eventos marcadoes de POC, com o objetivo de, durante a avaliação do modelo, não gerar overfiting.

## Libraries

In [23]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.sql import HiveContext 
hive_context = HiveContext(sc)
# para plots
%matplotlib inline

## constantes

In [24]:
mediana = 50  # mediana da ocorrência de eventos diários (feature)
percentil_max = 95 # percentil máximo considerado para treinar o modelo
D_90 = 90 # 90 dias a subtrair (usado somente para treino inicial)
D_1 = 1 # obtem eventos de D-1 para previsão de POC em produção
D_60 = 60 # obtem eventos a partir de D-60 para trás para treino constante do modelo em produção
D_2 = 2 # obtem eventos a partir de D-2 para trás para treino constante do modelo em produção

## Carga de Dados Brutos Hadoop Produção

In [53]:
####################################################################################
# PARA RODAR EM HADOOP PRODUCAO DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO MAC OSX 
####################################################################################
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')
poc_in = '/user/x181472/export/BASE_POC_ATM_090816.csv'
tranlog_in = hive_context.table('default.ext_juntatudo12')
tabelao_out = '/temporario/tabelao-temp-V5.2.5.csv'
tabelao_in = '/user/x184166/tabelao-temp-V5.2.5.csv' # Hermann
poceven_out = '/temporario/pokeventos-V2.csv'

df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')

root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



Numero de linhas carregadas: 18341921


In [26]:
#df_eventlog.limit(10).toPandas()

## Carga de dados Brutos MAC

In [27]:
####################################################################################
# PARA RODAR MAC OSX DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO HADOOP PRODUCAO 
####################################################################################
#eventlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tbfullog.csv'
#poc_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/BASE_POC_ATM_090816.csv'
#tranlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/dados/juntatudo11pordia-tranfin.csv'
#tabelao_out = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#tabelao_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#previsao = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/previsao-V5.2.2.csv'
#poceven_out = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/pokeventos.csv'

#funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
#funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

#rdd_eventlog = sc.textFile(eventlog_in)
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

#header = rdd_eventlog.first()

#df_eventlog = rdd_eventlog \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(",")) \
#    .map(lambda p: (p[1],p[2],p[3],p[4], p[5])) \
#    .toDF(['nr_sequ_even',
#            'dh_even', 
#            'cd_eqpt', 
#            'nr_idef_secu', 
#            'nr_eventos'
#                  ])
#df_eventlog.printSchema()    

In [28]:
#df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

In [29]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

df_eventlog.printSchema()   

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [17]:
#df_eventlog.limit(10).orderBy(col("dh_even").desc()).toPandas()

In [30]:
import datetime, time
from dateutil.relativedelta import relativedelta
# DEPOIS: acertar abaixo para pegar direto ultima data do arquivo original como data_ref

# obtendo faixa do período de histórico a partir da data de referencia
data_ref = datetime.datetime(2016, 8, 10) 
D_menos1 = data_ref + relativedelta(days=-D_1)
D_menos2 = data_ref + relativedelta(days=-D_2)
D_menos60 = data_ref + relativedelta(days=-D_60)
D_menos90 = D_menos60 + relativedelta(days=-D_90)
print "D-90:   " + str(D_menos90)
print "D-1 :   " + str(D_menos1)
print "D-2 :   " + str(D_menos2)
print "D-60 :   " + str(D_menos60)

D-90:   2016-03-13 00:00:00
D-1 :   2016-08-09 00:00:00
D-2 :   2016-08-08 00:00:00
D-60 :   2016-06-11 00:00:00


In [31]:
#data_ref = ("2016-05-10")
#data_D_menosl = ("2016-08-08")
dt_sql_90 = (time.mktime(datetime.datetime.strptime(str(D_menos90.date()), "%Y-%m-%d").timetuple()))
dt_sql_2 = (time.mktime(datetime.datetime.strptime(str(D_menos2.date()), "%Y-%m-%d").timetuple()))
dt_sql_60 = (time.mktime(datetime.datetime.strptime(str(D_menos60.date()), "%Y-%m-%d").timetuple()))
dt_sql_ref = (time.mktime(datetime.datetime.strptime(str(data_ref.date()), "%Y-%m-%d").timetuple()))
print dt_sql_2
print dt_sql_90
print dt_sql_60
print dt_sql_ref

1470625200.0
1457838000.0
1465614000.0
1470798000.0


In [32]:
# FILTRANDO SOMENTE ULTIMOS 30 DIAS do primeiro trimestre apenas
df_treinoevent = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
                                   .format(dt_sql_90,dt_sql_2)).orderBy("dh_even")
#                                .format(dt_sql_90,dt_sql_60)).orderBy("dh_even")
#df_eventlog = df_eventlog.where("CAST(dh_even AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
#df_treinoevent.limit(10).toPandas()

## Obtendo Eventos Marcadores de POC (selecionados como features)
Usando como baseline o arquivo de POCs reportados, serão identificados os eventos fora do padrão das agências/ATMs para serem usados como marcadores de POC no modelo preditivo.
São considerados fora do padrão, eventos que ocorrem em dias de ocorrência de POC, em agências reportadas como tendo ocorrência de POC, e que têm a quantidade de ocorrências no dia acima do percentil 75% da distribuição diária do eventos em todas as ATMs. Além disso, devem ser eventos que não aparecem com valores maiores que o percentil 75% nos dias em que não existem ocorrências de POCs.

In [33]:
# calculando percentis de baseline para todas as observações do log com base em agencia/atm/evento
# OBS: o uso do percentil desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem
df_eventlog_percen = df_treinoevent.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [34]:
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

In [35]:
rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

In [36]:
df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'mediana',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

In [37]:
df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [38]:
#df_eventlog.count()

In [39]:
#df_eventlog.limit(10).toPandas()

In [40]:
# colocar no log total as colunas de mediana e percentil máximo por agencia+atm+evento 
df_eventlog2 = df_treinoevent \
    .join(df_eventlog_percen, (df_treinoevent['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_treinoevent['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (df_treinoevent['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

In [41]:
#df_eventlog2.count()

In [42]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda
# testar s.replaceFirst("^0+(?!$)", "")

In [43]:
#df_eventlog2.orderBy('nr_sequ_even').limit(10).toPandas()

### Dados de registro de POC
Obtém os POCs de Retenção de Cartão, somente para os últimos 90 dias.
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

Identifica quais eventos que aconteceram nas agências e na data que saíram da faixa de percentil. estes eventos serão os **Marcadores de POC**.

In [44]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))



Número de linhas carregadas: 1019


In [45]:
#rdd_poc.take(2)

In [46]:
header = rdd_poc.first()
# pegando data inicial de comprometimento ao inves de data_cad, pois analise exploratoria mostrou que tem
# mais sentido usar esta data
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(",")) \
    .map(lambda p: (p[0],p[2],p[4], p[8])) \
    .toDF(['data_cad', 
            'atm',
            'agencia',
            'golpe'
        ])

df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

df_poc = df_poc \
    .withColumn('atm', df_poc["atm"].cast(StringType()))\
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
df_poc = df_poc.where("CAST(data_cad AS INT) BETWEEN {0} AND {1}"\
                      .format(dt_sql_90,dt_sql_2)).orderBy("data_cad")
#                                .format(dt_sql_90,dt_sql_60)).orderBy("data_cad")
#df_eventlog = df_eventlog.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
df_poc.limit(10).toPandas()


data_cad atm agencia                       golpe
0 2016-03-13        0990    GOLPE RETENTOR DE CARTAO
1 2016-03-13        0656                            
2 2016-03-13        0921  GOLPE RETENTOR DE ENVELOPE
3 2016-03-13        0140  GOLPE RETENTOR DE ENVELOPE
4 2016-03-13        4780    GOLPE RETENTOR DE CARTAO
5 2016-03-13        1561    GOLPE RETENTOR DE CARTAO
6 2016-03-14        3777  GOLPE RETENTOR DE ENVELOPE
7 2016-03-14        4691  GOLPE RETENTOR DE ENVELOPE
8 2016-03-14        0560  GOLPE RETENTOR DE ENVELOPE
9 2016-03-14        0733  GOLPE RETENTOR DE ENVELOPE

In [47]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_poc = df_poc\
    .withColumn('atm', trim(df_poc.atm))\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zero `esquerda
#df_poc.limit(10).toPandas()

## Selecionando Eventos por ATM/Agência em dias POC

In [48]:
# SELECIONANDO DO LOG TOTAL SOMENTE AGENCIAS E DIAS DE OCORRENCIA DE POC ACIMA DO PERCENTIL
df_poceven = df_eventlog2 \
    .join(df_poc, (df_poc['data_cad'] == df_eventlog2['dh_even']) &
                (df_poc['agencia'] == df_eventlog2['nr_idef_secu']), 'left')
#df_poceven.limit(10).toPandas()

In [49]:
# LOG SOMENTE COM AGENCIAS POC NAS DATAS POC E MAIOR QUE PERCENTIL
df_pocevenSIM = df_poceven\
    .filter((col("nr_eventos") > col('percentil_max')) &
            ((col("agencia") == col('nr_idef_secu'))))
    
#df_poceven2.limit(10).toPandas()

## Selecionando Eventos por ATM/Agência Fora de dias POC

In [50]:
# LOG SOMENTE COM TODAS AS AGENCIAS NAS DATAS FORA DE POC E MENOR QUE PERCENTIL
df_pocevenNAO = df_poceven\
    .filter((col("nr_eventos") <= col('percentil_max')) &
            (col('agencia').isNull()))
#    .filter(col('agencia').isNull())
#df_pocevenNAO.limit(10).toPandas()

In [51]:
# JOIN DAS DUAS ACIMA POR EVENTO PARA OBTER LISTA DE POKEVENTOS
df_pokeventos = df_pocevenSIM\
    .join(df_pocevenNAO,
          (df_pocevenSIM['nr_sequ_even'] == df_pocevenNAO['nr_sequ_even']), 'inner')\
    .drop(df_pocevenNAO["nr_sequ_even"])\
    .groupBy('nr_sequ_even').count()
    
#df_pokeventos.limit(10).toPandas()
#    .drop(df_eventlog_percen["nr_idef_secu"])\
#    .drop(df_eventlog_percen["cd_eqpt"])\
#    .drop(df_eventlog_percen["nr_sequ_even"])

In [52]:
l_evenpoc = df_pokeventos.toPandas().nr_sequ_even.tolist()
l_evenpoc

[u'4266',
 u'899',
 u'2034',
 u'4267',
 u'4268',
 u'2982',
 u'3260',
 u'4269',
 u'1642',
 u'5495',
 u'1599',
 u'3425',
 u'5498',
 u'1648',
 u'2422',
 u'676',
 u'3812',
 u'679',
 u'723',
 u'724',
 u'2761',
 u'3814',
 u'6776',
 u'2267',
 u'6778',
 u'3043',
 u'2767',
 u'5278',
 u'569',
 u'2492',
 u'4274',
 u'2493',
 u'1765',
 u'791',
 u'1650',
 u'1651',
 u'1769',
 u'1813',
 u'1653',
 u'2996',
 u'798',
 u'799',
 u'1656',
 u'682',
 u'1657',
 u'2430',
 u'3321',
 u'1658',
 u'5104',
 u'684',
 u'2433',
 u'2434',
 u'5558',
 u'6944',
 u'2930',
 u'5559',
 u'6332',
 u'688',
 u'5109',
 u'6333',
 u'689',
 u'6334',
 u'6336',
 u'6787',
 u'4556',
 u'6338',
 u'3666',
 u'4557',
 u'6339',
 u'2163',
 u'3829',
 u'3217',
 u'2165',
 u'3056',
 u'3551',
 u'4109',
 u'5171',
 u'3552',
 u'5172',
 u'626',
 u'3554',
 u'627',
 u'4284',
 u'628',
 u'4609',
 u'629',
 u'3442',
 u'1661',
 u'3894',
 u'5720',
 u'3895',
 u'5721',
 u'2392',
 u'3283',
 u'3896',
 u'3446',
 u'3898',
 u'3286',
 u'3899',
 u'5563',
 u'3288',
 u'6500

In [54]:
len(l_evenpoc)

342

In [56]:
# salvando a lista de pocs para ser usada no treino e uso do modelo
import csv
poceven_file = open(poceven_out, 'wb')
wr = csv.writer(poceven_file, quoting=csv.QUOTE_ALL)
wr.writerow(l_evenpoc)